In [1]:
import random
import numpy as np
import collections
from tqdm import tqdm
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import rl_utils
import GridWorld_v4
from draw import draw  # 绘图函数

In [2]:
class ReplayBuffer:
    """经验回放池"""

    def __init__(self, capacity):
        self.buffer = set()
        # self.buffer = collections.deque(maxlen=capacity)  # 队列,先进先出

    def add(self, state, action, reward, next_state, done):  # 将数据加入buffer
        # self.buffer.append((state, action, reward, next_state, done))
        self.buffer.add((state, action, reward, next_state, done))

    def sample(self, batch_size):  # 从buffer中采样数据,数量为batch_size
        transitions = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = zip(*transitions)
        return np.array(state), action, reward, np.array(next_state), done

    def size(self):  # 目前buffer中数据的数量
        return len(self.buffer)

In [3]:
class Qnet(torch.nn.Module):
    """只有一层隐藏层的Q网络"""

    def __init__(self, state_dim, hidden_dim, action_dim):
        super(Qnet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))  # 隐藏层使用ReLU激活函数
        return self.fc2(x)

In [4]:
class DQN:
    """DQN算法"""

    def __init__(
        self,
        state_dim,
        hidden_dim,
        action_dim,
        learning_rate,
        gamma,
        epsilon,
        target_update,
        device,
    ):
        self.action_dim = action_dim
        self.q_net = Qnet(state_dim, hidden_dim, self.action_dim).to(device)  # Q网络
        # 目标网络
        self.target_q_net = Qnet(state_dim, hidden_dim, self.action_dim).to(device)
        # 使用Adam优化器
        self.optimizer = torch.optim.Adam(self.q_net.parameters(), lr=learning_rate)
        self.gamma = gamma  # 折扣因子
        self.epsilon = epsilon  # epsilon-贪婪策略
        self.target_update = target_update  # 目标网络更新频率
        self.count = 0  # 计数器,记录更新次数
        self.device = device

    def take_action(self, state):  # epsilon-贪婪策略采取动作
        if np.random.random() < self.epsilon:
            action = np.random.randint(self.action_dim)
        else:
            state = torch.tensor([state], dtype=torch.float).to(self.device)
            # print(state)
            action = self.q_net(state).argmax().item()
            # print(action)
        return action

    def update(self, transition_dict):
        states = torch.tensor(transition_dict["states"], dtype=torch.float).to(
            self.device
        )
        actions = torch.tensor(transition_dict["actions"]).view(-1, 1).to(self.device)
        rewards = (
            torch.tensor(transition_dict["rewards"], dtype=torch.float)
            .view(-1, 1)
            .to(self.device)
        )
        next_states = torch.tensor(
            transition_dict["next_states"], dtype=torch.float
        ).to(self.device)
        dones = (
            torch.tensor(transition_dict["dones"], dtype=torch.float)
            .view(-1, 1)
            .to(self.device)
        )

        q_values = self.q_net(states).gather(1, actions)  # Q值
        # 下个状态的最大Q值
        max_next_q_values = self.target_q_net(next_states).max(1)[0].view(-1, 1)
        q_targets = rewards + self.gamma * max_next_q_values * (1 - dones)  # TD误差目标
        dqn_loss = torch.mean(F.mse_loss(q_values, q_targets))  # 均方误差损失函数
        self.optimizer.zero_grad()  # PyTorch中默认梯度会累积,这里需要显式将梯度置为0
        dqn_loss.backward()  # 反向传播更新参数
        self.optimizer.step()

        if self.count % self.target_update == 0:
            self.target_q_net.load_state_dict(self.q_net.state_dict())  # 更新目标网络
        self.count += 1

In [5]:
env_name = "CartPole-v0"

In [6]:
rows = 5  # 记得行数和列数这里要同步改
columns = 5
env = GridWorld_v4.GridWorld_v4(
    forbiddenAreaScore=-10, score=10, desc=[".....", ".##..", "..#..", ".#T#.", ".#..."]
)
env.show()
trajectorySteps = 1000

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️


In [7]:
lr = 0.0015
num_episodes = 500
hidden_dim = 128
gamma = 0.9
# epsilon = 0.01
epsilon = 1
target_update = 10
buffer_size = 10000
minimal_size = 16
batch_size = 8
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


random.seed(0)
np.random.seed(0)
# env.seed(0)
torch.manual_seed(0)
replay_buffer = ReplayBuffer(buffer_size)
# state_dim = env.observation_space.shape[0]
# action_dim = env.action_space.n

state_dim = 2
action_dim = 5
agent = DQN(
    state_dim, hidden_dim, action_dim, lr, gamma, epsilon, target_update, device
)

return_list = []
for i in range(100):
    with tqdm(total=int(num_episodes / 10), desc="Iteration %d" % i) as pbar:
        print(replay_buffer.size())
        if epsilon > 0.05:
            epsilon -= 0.005
        agent.epsilon = epsilon
        for i_episode in range(int(num_episodes / 10)):
            episode_return = 0
            state = env.reset()
            done = False
            while not done:
                # print(1)
                action = agent.take_action((state // 5, state % 5))
                next_state, reward, done, _ = env.step(action)
                replay_buffer.add(
                    (state // 5, state % 5),
                    action,
                    reward,
                    (next_state // 5, next_state % 5),
                    done,
                )
                state = next_state
                episode_return += reward
                # 当buffer数据的数量超过一定值后,才进行Q网络训练
                if replay_buffer.size() > minimal_size:
                    b_s, b_a, b_r, b_ns, b_d = replay_buffer.sample(batch_size)
                    transition_dict = {
                        "states": b_s,
                        "actions": b_a,
                        "next_states": b_ns,
                        "rewards": b_r,
                        "dones": b_d,
                    }
                    agent.update(transition_dict)
            # print(6)
            return_list.append(episode_return)
            if (i_episode + 1) % 10 == 0:
                pbar.set_postfix(
                    {
                        "episode": "%d" % (num_episodes / 10 * i + i_episode + 1),
                        "return": "%.3f" % np.mean(return_list[-10:]),
                    }
                )
            pbar.update(1)

    print("epsilon:", epsilon)
    s = np.array([[i, j] for i in range(5) for j in range(5)])
    states = torch.tensor(s, dtype=torch.float32).to(device)

    action_values = agent.target_q_net(states).cpu().view(-1, 5).detach().numpy()

    greedy_policy = np.argmax(action_values, axis=1)

    one_hot_policy = np.eye(5)[greedy_policy]  # 把传入的一维列表，转成二维one-hot型

    # p1是目标方向的概率，p0是另外四个方向的概率
    p1 = 1 - epsilon * (4 / 5)
    p0 = epsilon / 5
    d = {1: p1, 0: p0}
    policy_epsilon = np.vectorize(d.get)(one_hot_policy)  # 生成一个epsilon的policy

    state_values = np.sum(policy_epsilon * action_values, axis=1)

    draw(state_values.reshape(5, 5), greedy_policy)

Iteration 0:   0%|          | 0/50 [00:00<?, ?it/s]

0


TypeError: Population must be a sequence.  For dicts or sets, use sorted(d).

In [ ]:
episodes_list = list(range(len(return_list)))
plt.plot(episodes_list, return_list)
plt.xlabel("Episodes")
plt.ylabel("Returns")
# plt.title('DQN on {}'.format(env_name))
plt.show()

mv_return = rl_utils.moving_average(return_list, 9)
plt.plot(episodes_list, mv_return)
plt.xlabel("Episodes")
plt.ylabel("Returns")
# plt.title('DQN on {}'.format(env_name))
plt.show()

In [ ]:
s1 = set()

In [ ]:
type(s1)